In [1]:
# macro
fileName = r'/home/liang.liang/code/Tools/python/pycharts/2019CW15.4.xlsx'
reportName = '20190411.html'
CWsMonitor = 15

TotalFeatures = 1024


In [2]:
# import lib

from pyecharts.engine import create_default_environment
from pyecharts import Page
from pyecharts import Bar, Line, Grid
from pyecharts import Liquid

import xlrd

In [3]:
data = xlrd.open_workbook(fileName)

table = data.sheets()[0]


Data =  {"KeyID":table.col_values(2,1),
         "funcGroup":table.col_values(4,1),
         "status":table.col_values(5,1),
         "group":table.col_values(6,1),
         "targetCW":table.col_values(7,1)

        }



In [4]:
# define calss for project management

In [5]:
class Features:
    """it's 37w project"""
    
    __group  = ("Audio","Media","Radio","Online","HMI","SDS","AndroidApp","AndroidOS"
                   ,"Update","Navi","System","CarConnect","DVR","PCC")    
    
    __status = ("New","In Analysis","Pending","In Implementation","In Testing","Released")
    
    __cw = [ ("CW%02d" %i)  for i in range(1,(CWsMonitor+1))]

    
    __featureLen = len(Data["KeyID"])
    
    __autowidth = 1200
    
    __countReleased = Data["status"].count("Released")
    
    
    def __init__(self,source,export):
        self.__source = source
        self.__export = export    
 
        self.__status_cw = {}
        for i in self.__status:
            self.__status_cw["%s" %i] = {c:0 for c in self.__cw }
            
        for s in self.__status:
            for c in self.__cw:
                temp_count = 0
                for i in range(len(Data["KeyID"])):
                    if(Data["status"][i] == s and Data["targetCW"][i] == c[2:] ):
                        temp_count +=1
                self.__status_cw[s][c] = temp_count
        

        self.__status_group_delay = {}
        for i in self.__status:
            self.__status_group_delay["%s" %i] = {j:0 for j in self.__group}


        for s in self.__status:
            for g in self.__group:
                temp_count = 0
                for ii in range(len(Data["KeyID"])):
                    if (Data["status"][ii] == s and Data["group"][ii] == g ):
                        temp_count +=1
        
                self.__status_group_delay[s][g] = temp_count
    
        
    def GetExportFileName(self):
        return self.__export
        
        
    def ProjectFinishRate(self):
        liquid = Liquid(title="37W Project", title_text_size = 20
              ,subtitle = "Features Completion Rate")
        liquid.add("Liquid", [self.__countReleased/TotalFeatures],width = self.__autowidth/2)
        
        return liquid


    
    def CWStatus(self):
        barCW = Bar(title="Feature Status", title_text_size = 20
          ,subtitle ="Total items:" +str(self.__featureLen)
          ,width = self.__autowidth)

        for i in self.__status:
            barCW.add(i, self.__cw, list(self.__status_cw[i].values())
            ,xaxis_rotate=0,yaxis_rotate=0,xaxis_interval= 0
            ,is_label_show=False,yaxis_pos = "left"
            ,xaxis_type = 'category',is_stack = True,is_convert = True,yaxis_label_textsize = 12
            ,yaxis_name_gap = 5
            ,is_toolbox_show = True,is_more_utils=True
            ,yaxis_interval = 0,yaxis_margin= 0
            ,legend_pos="30%"
           )
        
        return barCW
    
    
    def GroupStatus(self):
        bar1 = Bar(title="Feature Status", title_text_size = 20
          ,subtitle ="Current status for each group"
          ,width = self.__autowidth)

        for i in self.__status:
      
            bar1.add(i,self.__group, list(self.__status_group_delay[i].values())
            ,xaxis_rotate=30,yaxis_rotate=0,xaxis_interval= 0
            ,is_label_show=False,yaxis_pos = "left"
            ,xaxis_type = 'category',is_stack = True,is_convert = False,yaxis_label_textsize = 12
            ,yaxis_name_gap = 5
            ,is_toolbox_show = True,is_more_utils=True
            ,yaxis_interval = 0,yaxis_margin= 0
            ,legend_pos="30%"
           )
 
        return bar1


In [6]:
features = Features(fileName,reportName)

In [7]:
features.ProjectFinishRate()

In [8]:
features.GroupStatus()

In [9]:
features.CWStatus()

In [10]:
#coding=utf-8
from __future__ import unicode_literals


grid1 = Grid()


grid1.add(features.ProjectFinishRate(), grid_left=  "10%")


grid2 = Grid()
grid2.add(features.GroupStatus(),grid_left=  "10%")


grid3 = Grid()
#grid3.add(barCW,grid_left=  "10%")

grid3.add(features.CWStatus(),grid_left=  "10%")



page = Page()         # step 1

page.add(grid1)
page.add(grid2)
page.add(grid3)


page.render(path = features.GetExportFileName())



In [11]:
# below is test

In [12]:
test_status = ("New","In Analysis","Pending","In Implementation","In Testing","Released")
test_group  = ("Audio","Media","Radio","Online","HMI","SDS","AndroidApp","AndroidOS"
                   ,"Update","Navi","System","CarConnect","DVR","PCC")